# Meta Programming

In [1]:
prog = "1 + 1"

"1 + 1"

In [4]:
ex1 = parse(prog)

:(1 + 1)

In [5]:
typeof(ex1)

Expr

In [7]:
(ex1.head, ex1.args, ex1.typ)

(:call,Any[:+,1,1],Any)

In [8]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [9]:
ex1 == ex2

true

The key point here is that Julia code is internally represented as a data structure that is accessible from the language itself.

In [10]:
dump(ex2)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


In [11]:
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [12]:
dump(ex3)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol /
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Int64 4
        3: Int64 4
      typ: Any
    3: Int64 2
  typ: Any


In [13]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

In [14]:
:foo == symbol("foo")

true

In [15]:
symbol(:var,'_',"sym")

:var_sym

In [17]:
wow = :(::)

:(::)

In [19]:
dump(wow)

Symbol ::


In [20]:
:(a+b*c+1)

:(a + b * c + 1)

In [21]:
:(a + b*c + 1)  ==
       parse("a + b*c + 1") ==
       Expr(:call, :+, :a, Expr(:call, :*, :b, :c), 1)

true

In [22]:
ex = quote
   x = 1
   y = 2
   x + y
end

quote  # In[22], line 2:
    x = 1 # In[22], line 3:
    y = 2 # In[22], line 4:
    x + y
end

In [23]:
a = 1
ex = :($a + b)

:(1 + b)

In [24]:
ex = :(a in $:((1,2,3)) )

:($(Expr(:in, :a, :((1,2,3)))))

In [25]:
dump(ex)

Expr 
  head: Symbol in
  args: Array(Any,(2,))
    1: Symbol a
    2: Expr 
      head: Symbol tuple
      args: Array(Any,(3,))
        1: Int64 1
        2: Int64 2
        3: Int64 3
      typ: Any
  typ: Any


In [26]:
eval(ex)

true

In [27]:
:(a in $:((1,2,3))) == :(a in $(:((1,2,3))))

true

In [28]:
function make_expr(op, opr1, opr2)
    opr1f, opr2f = map((opr1, opr2)) do x
        isa(x, Number) ? 2*x : x
    end
    
    Expr(:call, op, opr1f, opr2f)
end

make_expr (generic function with 1 method)

In [29]:
make_expr(:+, 2, :x)

:(4 + x)

In [30]:
macro sayhello(name)
   return :( println("Hello, ", $name) )
end

In [31]:
@sayhello "Julia"

Hello, Julia


In [32]:
macroexpand( :(@sayhello("human")) )

:(println("Hello, ","human"))

In [33]:
macro showarg(x)
   show(x)
   # ... remainder of macro, returning an expression
end

In [36]:
@showarg 1+2

:(1 + 2)

In [37]:
macroexpand(:(@assert a==b))

:(if a == b
        nothing
    else 
        Base.throw(Base.Main.Base.AssertionError("a == b"))
    end)

In [38]:
module MyModule

macro wrong_time(ex)
  return quote
    local t0 = time()
    local val = $ex
    local t1 = time()
    println("elapsed time: ", t1-t0, " seconds")
    val
  end
end

macro right_time(ex)
  return quote
    local t0 = time()
    local val = $(esc(ex))
    local t1 = time()
    println("elapsed time: ", t1-t0, " seconds")
    val
  end
end
end

MyModule

In [56]:
for op = (:×, :⋅, :∧)
    @eval ($op)(a,b) = @sprintf "(%s %s %s)" a string($op) b
    @eval ($op)(a,b,c) = ($op)(($op)(a,b),c)
end
(2 ⋅ 3, 2 × 3, 2 ∧ 3)

(6,"(2 cross 3)","(2 ∧ 3)")

LoadError: LoadError: UndefVarError: op not defined
while loading In[46], in expression starting on line 1